In [2]:
! pip install pymysql
import pymysql.cursors
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 584.6 kB/s eta 0:00:00


In [3]:
host = 'den1.mysql6.gear.host'
db = 'situation'
usr = 'situation'
psword = input()

aichem567.


In [4]:
# database connection
connection = pymysql.connect(
    host=host,
    user=usr,
    password=psword,
    database=db,
    charset="utf8mb4",
    cursorclass=pymysql.cursors.DictCursor,
           )
cursor = connection.cursor()

UserId = 'zzyyliu98'
sql ="SELECT * FROM situation.a1_results;"

    # with connection.cursor() as cursor:
    #   cursor.execute(sql % UserId)

cursor.execute(sql)
        # result for searching Databank
result = cursor.fetchall()
        # change result to dataframe
df_raw = pd.DataFrame(result, columns=[i[0] for i in cursor.description])

print(df_raw)

    ExperimentNumber  Weight  Yield                SampleDesc  QualityTest  \
0                  1     297   78.0   White crystalline solid            1   
1                  2     488   70.0    Solid formation varies            1   
2                  3     315   80.0  Speed of reaction varies            1   
3                  4     120    NaN   Purity by melting point            0   
4                  5     232   70.0      Degree of completion            1   
..               ...     ...    ...                       ...          ...   
56                 3      90   66.0                Compound B            0   
57                 3     130   93.0                Compound C            1   
58                 3     170   94.0                Compound A            1   
59                 3      85   65.0                Compound B            0   
60                 3     125   95.0                Compound C            1   

    Spectrum_ID  ReactionTime  
0           NaN           NaN  

From example: MLPClassifier

In [5]:
df_raw1 = df_raw

#Select all results in Experiment 3 and delete the results with no reaction time
df_raw1 = df_raw1[df_raw1['ExperimentNumber'] == 3]
df_raw1 = df_raw1.dropna(subset=['ReactionTime'])


# Define features and target variable
X = df_raw1.drop(columns=["ExperimentNumber", "QualityTest","SampleDesc",
            "Spectrum_ID"])
y = df_raw1["QualityTest"]

X = pd.get_dummies(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = MLPClassifier(random_state=42, max_iter=500)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {round(accuracy * 100, 4)} %\n\nReport:\n{report}")


Accuracy: 40.0 %

Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.40      1.00      0.57         4

    accuracy                           0.40        10
   macro avg       0.20      0.50      0.29        10
weighted avg       0.16      0.40      0.23        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Suggestion: Neural Network Regression（Keras）

In [6]:
# keras model
model_keras= Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model_keras.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# training
model_keras.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

# prediction
y_pred = model_keras.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)


# results
test_results = X_test.copy()
test_results['Predicted_QualityTest'] = y_pred_classes.flatten()

print(f'Accuracy: {accuracy}')
print(test_results)

Epoch 1/50
2/2 [==============================] - 2s 378ms/step - loss: 95.6398 - accuracy: 0.7368 - val_loss: 81.1182 - val_accuracy: 0.4000
Epoch 2/50
2/2 [==============================] - 0s 81ms/step - loss: 32.7415 - accuracy: 0.7368 - val_loss: 23.8119 - val_accuracy: 0.6000
Epoch 3/50
2/2 [==============================] - 0s 50ms/step - loss: 55.4942 - accuracy: 0.2632 - val_loss: 0.9891 - val_accuracy: 0.9000
Epoch 4/50
2/2 [==============================] - 0s 64ms/step - loss: 2.7784 - accuracy: 0.7105 - val_loss: 76.6378 - val_accuracy: 0.4000
Epoch 5/50
2/2 [==============================] - 0s 49ms/step - loss: 36.8784 - accuracy: 0.7368 - val_loss: 113.5561 - val_accuracy: 0.4000
Epoch 6/50
2/2 [==============================] - 0s 50ms/step - loss: 49.4608 - accuracy: 0.7368 - val_loss: 115.0064 - val_accuracy: 0.4000
Epoch 7/50
2/2 [==============================] - 0s 66ms/step - loss: 49.6066 - accuracy: 0.7368 - val_loss: 96.3196 - val_accuracy: 0.4000
Epoch 8/50
2